# Data Gathering

## Data Download

Data from the Bureau of Transpotation Statistics with a Python script. It was downloaded in monthy chunks from Oct. 1987 - Mar. 2023. In total, the uncompressed files are ~94GB.

Copy of script in cell below and published on my GitHub.

```
import requests
import certifi

baseURL = 'https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_'

for i in range(1987, 2023): #grab years from 1987-2022
    for j in range(1, 13): #grab months from Jan.-Dec.
        specificURL = baseURL + str(i) + '_' + str(j) + ".zip" #add year and month to make full URL
        response = requests.get(specificURL, stream=True) #get file from specified web address
        filename = "BTS_" + str(j) + '_' + str(i) + ".zip" #make filename from year and month
        with open(filename, "wb") as f:
            f.write(response.content) #write it to a file
```

Note: Modification of certifi install is required to have valid SSL certificates for requesting the download from TranStats

## Data Ingest Trial 1

The initial attempt was to load all the data into Pandas dataframes and concatenate them together. I knew it probably wouldn't work (especially b/c Pandas has an upper limit of 100GB per dataframe and I was too close to that). Result was computer ran out of memory and swap and the Python kernel crashed.

In [1]:
import pandas as pd

```
#This code block iterates through all filepaths to the csv files,
# creates a dataframe from that csv file,
# and appends the dataframe to the list of dataframes
dfList = list()

#Latin-1 encoding used because there was an error with default (UTF-8) encoding

for month in range(10,13):
    path = '/Users/aidencamilleri/Library/CloudStorage/OneDrive-CollegeofCharleston/Personal Projects/Code Projects/BTSDownload/Unzipped/1987/BTS_' + str(month) + '_1987/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_1987_' + str(month) + '.csv'
    dfList.append(pd.read_csv(path, encoding = 'latin-1'))
    
for year in range(1988, 2023):
    for month in range(1, 13):
        path = '/Users/aidencamilleri/Library/CloudStorage/OneDrive-CollegeofCharleston/Personal Projects/Code Projects/BTSDownload/Unzipped/' + str(year) + '/BTS_' + str(month) + '_' + str(year) + '/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_' + str(year) + '_' + str(month) + '.csv'
        dfList.append(pd.read_csv(path, encoding = 'latin-1'))
        
for month in range(1,4):
    path = '/Users/aidencamilleri/Library/CloudStorage/OneDrive-CollegeofCharleston/Personal Projects/Code Projects/BTSDownload/Unzipped/2023/BTS_' + str(month) + '_2023/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_' + str(month) + '.csv'
    dfList.append(pd.read_csv(path, encoding = 'latin-1'))
```

```
megaDF = pd.concat(dfList)
```

## Data Ingest Trial 2

Now I am attempting to import the data into a local Microsoft SQL Server. As I am running this project on a personal MacBook, the MSSQL Server is being hosted in a Docker container. I am having issues importing the individual CSV files into the MSSQL Server because of formatting quirks of the CSV files from the BTS.

In [ ]:
import pyodbc